#Preparing Data

In [4]:
from zipfile import ZipFile
with ZipFile("data.zip","r") as zip_ref:
  zip_ref.extractall("data")

In [5]:
import numpy as np
np.random.seed(5) 
import tensorflow as tf
#tf.set_random_seed(2)
import matplotlib.pyplot as plt
%matplotlib inline
import os
import cv2
import keras
from sklearn.model_selection import train_test_split

train_dir = 'data/train/train'
eval_dir = 'data/train/test'


#2. Loading the data
#Helper function to load images from given directories
def load_images(directory,uniq_labels):
    images = []
    labels = []
    for idx, label in enumerate(uniq_labels):
        if (len(uniq_labels)>1):
            for file in os.listdir(directory + "/" + label):
                filepath = directory + "/" + label + "/" + file
                #image = cv2.resize(cv2.imread(filepath), (64, 64))
                image = cv2.imdecode(np.fromfile(filepath, dtype=np.uint8), cv2.IMREAD_UNCHANGED)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                image = cv2.resize(image, (64, 64))
                images.append(image)
                labels.append(idx)
        else:
            filepath = directory + "/" + label 
            #image = cv2.resize(cv2.imread(filepath), (64, 64))
            image = cv2.imdecode(np.fromfile(filepath, dtype=np.uint8), cv2.IMREAD_UNCHANGED)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            image = cv2.resize(image, (64, 64))
            images.append(image)
            labels.append(idx)
    images = np.array(images)
    labels = np.array(labels)
    return(images, labels)

In [6]:
CATEGORIES = sorted(os.listdir(train_dir))
#read images in train folder
images, labels = load_images(directory = train_dir, uniq_labels = CATEGORIES)
#read images in test folder


In [7]:
CATEGORIES

['A', 'B', 'C', 'F', 'G', 'L', 'M', 'O', 'Q', 'V', 'Y', 'nothing']

In [8]:
X_eval, y_eval = load_images(directory = eval_dir, uniq_labels = sorted(os.listdir(eval_dir)))

In [9]:
#split data to train and test
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size = 0.1, stratify = labels)

n = len(sorted(os.listdir(train_dir)))
train_n = len(X_train)
test_n = len(X_test)
# eval_n = len(X_eval)

print("Total number of symbols: ", n)
print("Number of training images: " , train_n)
print("Number of testing images: ", test_n)
# print("Number of evaluation images: ", eval_n)

Total number of symbols:  12
Number of training images:  7894
Number of testing images:  878


In [10]:
X_train.shape



(7894, 64, 64)

In [11]:
#preprocessing
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)
y_eval = tf.keras.utils.to_categorical(y_eval)

X_train = X_train.astype('float32')/255.0
X_test = X_test.astype('float32')/255.0
X_eval = X_eval.astype('float32')/255.0
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

In [12]:
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Conv2D, Dense, Dropout, Flatten
from keras.layers import Flatten, Dense
from keras.models import Sequential
#build the model
model = Sequential()
model.add(Conv2D(filters = 64, kernel_size = 5, padding = 'same', activation = 'relu', input_shape = (64, 64, 1)))
model.add(Conv2D(filters = 64, kernel_size = 5, padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (4, 4)))
model.add(Dropout(0.5))
model.add(Conv2D(filters = 128 , kernel_size = 5, padding = 'same', activation = 'relu'))
model.add(Conv2D(filters = 128 , kernel_size = 5, padding = 'same', activation = 'relu'))
model.add(MaxPooling2D(pool_size = (4, 4)))
model.add(Dropout(0.5))
model.add(Conv2D(filters = 256 , kernel_size = 5, padding = 'same', activation = 'relu'))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(12, activation='softmax'))#12 classes

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 64)        1664      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        102464    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       204928    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 128)       409728    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 4, 4, 128)         0

In [14]:
#compil the model
model.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])
#fit the model
hist = model.fit(X_train, y_train, epochs = 5, batch_size = 64)

Epoch 1/5
124/124 [==============================] - 44s 79ms/step - loss: 2.3541 - accuracy: 0.2266
Epoch 2/5
124/124 [==============================] - 9s 74ms/step - loss: 0.9829 - accuracy: 0.6691
Epoch 3/5
124/124 [==============================] - 9s 74ms/step - loss: 0.4092 - accuracy: 0.8608
Epoch 4/5
124/124 [==============================] - 9s 74ms/step - loss: 0.2507 - accuracy: 0.9158
Epoch 5/5
124/124 [==============================] - 9s 74ms/step - loss: 0.1501 - accuracy: 0.9506


In [15]:
#save model
model.save('ASLGray_model.h5')

#load model
#model=tf.keras.models.load_model('ASLGray.model',save_format='h5')

In [16]:
#Download model from kaggle
#from IPython.display import FileLinks
#FileLinks(r'ASLGray.model')

In [17]:
#!zip -r saved_model.zip 'ASLGray_model.h5'

In [18]:
#Accuracy of model
score = model.evaluate(x = X_test, y = y_test, verbose = 0)
print('Accuracy for test images:', round(score[1]*100, 3), '%')

Accuracy for test images: 97.153 %


In [19]:
#prepare image to prediction
def prepare(filepath):
    image = cv2.imdecode(np.fromfile(filepath, dtype=np.uint8), cv2.IMREAD_UNCHANGED)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    image = cv2.resize(image, (64, 64))
    image=image.reshape(-1, 64, 64, 1)
    image=image.astype('float32')/255.0
    return  image

import scipy
import tensorflow as tf
import torch
#use this function to predict images
def predict(my_model, filepath):
    prediction = model.predict([prepare(filepath)])
    probs = scipy.special.softmax(prediction[0])
    confidence=round((1-max(probs))*100,5) 
    category = np.argmax(prediction[0])
    print("The image class is: " + str(CATEGORIES[category]))
    print("Confidence: ",confidence," %")

In [20]:
i=1
for file in os.listdir(eval_dir+"/B"):
  predict(model,eval_dir+"/B/"+file)
  if i==1:
    break
  


The image class is: A
Confidence:  85.33945  %
